# Домашнее задание №2

### Тани Казаковой

### Устанавливаем проверку PEP-8.

In [1]:
! pip3 install pycodestyle flake8 pycodestyle_magic

In [2]:
%load_ext pycodestyle_magic
%pycodestyle_on

### Обрабатываем книгу с помощью Mystem

#### Устанавливаем

In [3]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [4]:
from pymystem3 import Mystem

#### Читаем файл

In [5]:
with open('oblomov_goncharov.txt', encoding='utf-8') as f:
    book = f.read()
# print(book)  # проверить, что текст на месте

####  Обрабатываем

In [6]:
m = Mystem()

In [7]:
from pprint import pprint

In [8]:
import time
start = time.time()
razb_mys = m.analyze(book)
end = time.time()
print(end - start)
# pprint(razb_mys)  # посмотреть, что получилось

15.443830728530884


#### Переводим в JSON

In [9]:
import json
with open('razbor_mys.json', 'w', encoding='utf-8') as f:
    for a_razb in razb_mys:
        json.dump(a_razb, f, ensure_ascii=False)  # переводим в JSON
        f.write('\n')

### Обрабатываем книгу с помощью Pymorthy

#### Устанавливаем pymorthy и ntkl

In [10]:
pip install pymorphy2

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [12]:
import nltk

In [13]:
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')

#### Разбиваем текст на слова

In [14]:
tok_book = word_tokenize(book)

In [15]:
# Кажется, не пригодилось
from nltk.corpus import stopwords
sw = stopwords.words('russian')  # загружаем нужный список стоп-слов

In [16]:
# приводим к нижнему регистру и оставляем только последовательности из букв
words = [w.lower() for w in tok_book if w.isalpha()]
# print(words)  # посмотреть, что получилось

#### Разбираем

In [17]:
parsed_words = []
start = time.time()
for w in words:
    parsed_words.append(morph.parse(w)[0])
end = time.time()
print(end - start)
# pprint(parsed_words)  # посмотреть, что получилось

48.51103115081787


#### Переводим в JSON

In [18]:
with open('razbor_pym.json', 'w', encoding='utf-8') as f:
    for w_razb in parsed_words:
        w_to_json = {'word': w_razb.word,
                     'lemma': w_razb.normal_form,
                     'part': w_razb.tag.POS}
        json.dump(w_to_json, f, ensure_ascii=False)
        f.write('\n')

### Ответы на вопросы

#### Какую долю слов составляет каждая часть речи?

In [19]:
j_razbor_pym = []
for line in open('razbor_pym.json'):
    j_razbor_pym.append(json.loads(line))

In [20]:
import collections
POS_razb = []
for razb_p in j_razbor_pym:
    if "part" in razb_p:
        POS_razb.append(razb_p['part'])
counter = collections.Counter(POS_razb)
# получили кол-во слов каждой части речи
# print(counter)  # посмотреть, что получилось

for part in counter:
    dol = str(counter[part]/len(j_razbor_pym))
    print('Доля ' + str(part) + ' - ' + dol + '.')
    # None - для слов без части речи

Доля NOUN - 0.2351339773088457.
Доля ADJF - 0.08240399098987052.
Доля VERB - 0.1516175393502204.
Доля CONJ - 0.11666873954921714.
Доля ADVB - 0.0663253320066885.
Доля ADJS - 0.009860011331757942.
Доля PREP - 0.10021696170694969.
Доля PRCL - 0.07032599532910465.
Доля PRTF - 0.006149551566408247.
Доля None - 0.0015062946533449416.
Доля NUMR - 0.005610602103284828.
Доля NPRO - 0.0992703453422329.
Доля INFN - 0.027873360695383.
Доля GRND - 0.010433508837389274.
Доля COMP - 0.0036897309398449484.
Доля PRTS - 0.0019415999889446263.
Доля PRED - 0.007648936611251607.
Доля INTJ - 0.0033235216892610864.


#### Найдите топ-20 глаголов

In [21]:
VERB_razb = []
for razb_p in j_razbor_pym:
    if "part" in razb_p:
        if "lemma" in razb_p:
            if razb_p['part'] == 'VERB':
                VERB_razb.append(razb_p['lemma'])
verb_counter = collections.Counter(VERB_razb)
# print(verb_counter)  # посмотреть, что получилось
print('20 самых популярных глаголов: ')
i = 0
for verb in verb_counter.most_common():
    print(verb[0] + ' - ' + str(verb[1]/len(VERB_razb)))
    i += 1
    if i > 19:
        break

20 самых популярных глаголов: 
быть - 0.05792279998177095
сказать - 0.04147108417262908
говорить - 0.024973795743517296
спросить - 0.01886706466754774
мочь - 0.017044159868750856
знать - 0.01339835027115709
стать - 0.012076744292029349
хотеть - 0.011256437132570752
думать - 0.009205669233924258
смотреть - 0.008066353734676207
видеть - 0.008066353734676207
любить - 0.00747390967506722
отвечать - 0.007200473955247687
дать - 0.007063756095337921
прибавить - 0.00638016679578909
пойти - 0.006289021555849246
идти - 0.005651004876270337
делать - 0.0056054322563004145
начать - 0.0055142870163605705
жить - 0.005468714396390648


#### Найдите топ-20 наречий

In [22]:
ADVB_razb = []
for razb_p in j_razbor_pym:
    if "part" in razb_p:
        if "lemma" in razb_p:
            if razb_p['part'] == 'ADVB':
                ADVB_razb.append(razb_p['lemma'])
advb_counter = collections.Counter(ADVB_razb)
# print(advb_counter)  # посмотреть, что получилось
print('20 самых популярных наречий: ')
i = 0
for advb in advb_counter.most_common():
    print(advb[0] + ' - ' + str(advb[1]/len(ADVB_razb)))
    i += 1
    if i > 19:
        break

20 самых популярных наречий: 
потом - 0.04229607250755287
там - 0.032711740806333996
вдруг - 0.032607563287842485
опять - 0.03104490051046984
теперь - 0.026356912178351913
где - 0.024690071882487757
уж - 0.02010626106886134
тут - 0.019377018439420773
иногда - 0.016460047921658505
никогда - 0.01593916032920096
уже - 0.014584852588811335
почти - 0.014168142514845296
зачем - 0.011667882071049067
тоже - 0.01125117199708303
хорошо - 0.010834461923116991
тогда - 0.010521929367642463
здесь - 0.010417751849150954
вон - 0.010209396812167934
давно - 0.009896864256693406
наконец - 0.009688509219710387


### Получаем текст из лемм

In [23]:
text_to_lem = ' '.join(words)
lemmas = m.lemmatize(text_to_lem)
text_lemm = []
for lemma in lemmas:
    if lemma != ' ':
        if lemma != '\n':
            text_lemm.append(lemma)
# print(text_lemm)  # посмотреть, что получилось

### Топ-25 биграмм

In [24]:
bigr_l = list(nltk.bigrams(text_lemm))
bigr_counter = collections.Counter(bigr_l)
print('25 биграмм: ')
i = 0
for bigr in bigr_counter.most_common():
    print(bigr[0][0] + " " + bigr[0][1])
    i += 1
    if i > 24:
        break

25 биграмм: 
и не
илья ильич
он не
сказать он
я не
у он
не быть
он и
на он
как будто
не знать
что я
что он
он в
что ж
мочь быть
сказать она
она не
не мочь
у она
и в
сказать обломов
что это
ничто не
что ты


### Топ-25 триграмм

In [25]:
threegr_l = list(nltk.ngrams(text_lemm, 3))
threegr_counter = collections.Counter(threegr_l)
print('25 триграмм: ')
i = 0
for threegr in threegr_counter.most_common():
    print(threegr[0][0] + " " + threegr[0][1] + " " + threegr[0][2])
    i += 1
    if i > 24:
        break

25 триграмм: 
в самый дело
глядеть на он
что ж ты
не знать что
на другой день
что это за
до сей пора
сказать илья ильич
в первый раз
по крайний мера
у он в
смотреть на он
и тот же
что он не
он и не
и не знать
с тот пора
глядеть на она
что ж это
один и тот
что это такой
на выборгский сторона
илья ильич и
я ничто не
поглядеть на он
